In [ ]:
%pip install requests


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import requests

# FHIR server base URL
base_url = 'https://hapi.fhir.org/baseR4'

# Function to search for MedicationRequest resources by a given code
def get_medication_requests_by_code(medication_code):
    # Construct the query URL
    query_url = f"{base_url}/MedicationRequest?code={medication_code}&_include=MedicationRequest:subject"
    
    # Send the request to the FHIR API
    response = requests.get(query_url)
    
    # Return the JSON response if successful
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception("Error fetching medication requests: HTTP Status Code", response.status_code)

# Function to extract patient ids from the MedicationRequest search set
def get_patient_references(medication_requests):
    patient_references = []
    for entry in medication_requests.get('entry', []):
        resource = entry.get('resource', {})
        if resource.get('resourceType') == 'MedicationRequest' and 'subject' in resource:
            patient_references.append(resource['subject']['reference'])
    return patient_references

# Function to get patient Contact details including emails
def get_patient_emails(patient_references):
    patient_emails = []
    for reference in patient_references:
        patient_id = reference.split('/')[-1]
        query_url = f"{base_url}/Patient/{patient_id}"
        response = requests.get(query_url)
        if response.status_code == 200:
            patient = response.json()
            for contact in patient.get('telecom', []):
                if contact.get('system') == 'email':
                    patient_emails.append(contact.get('value'))
    return patient_emails

# Step 1: Get MedicationRequests by medication code
medication_code = "39623"
medication_requests = get_medication_requests_by_code(medication_code)
print(medication_requests)

# Step 2: Extract patient references from MedicationRequests
patient_references = get_patient_references(medication_requests)

# Step 3: Get patient emails from Patient resources
patient_emails = get_patient_emails(patient_references)

# Display the results
print(patient_emails)



{'resourceType': 'Bundle', 'id': 'e891a0ad-1ff8-4603-a586-788c3efbdd35', 'meta': {'lastUpdated': '2023-11-09T20:45:55.166+00:00'}, 'type': 'searchset', 'total': 0, 'link': [{'relation': 'self', 'url': 'https://hapi.fhir.org/baseR4/MedicationRequest?_include=MedicationRequest%3Asubject&code=39623'}]}
[]


In [1]:

# The endpoint for searching patients by surname
patient_search_url = f'{base_url}/Patient'

# The parameters for the search query, searching for patients with the surname 'Smith'
search_params = {
    'family': 'Smith'
}

# Make the GET request to search for patients with the surname 'Smith'
response = requests.get(patient_search_url, params=search_params)

# Check if the request was successful
if response.status_code == 200:
    # Parse the response as JSON
    patients = response.json()

    # Extract the email addresses from the patient data
    email_addresses = []
    for entry in patients.get('entry', []):
        patient = entry.get('resource')
        for contact in patient.get('telecom', []):
            if contact.get('system') == 'email':
                email_addresses.append(contact.get('value'))

    # Print out the email addresses
    for email in email_addresses:
        print(email)
else:
    print(f'Request failed with status code: {response.status_code}')

sarahsmith@gmail.com


In [2]:

# The endpoint for searching patients by surname
patient_search_url = f'{base_url}/Patient'

# The parameters for the search query, searching for patients with the surname 'Smith'
search_params = {
    'family': 'Smith'
}

# Make the GET request to search for patients with the surname 'Smith'
response = requests.get(patient_search_url, params=search_params)

# Check if the request was successful
if response.status_code == 200:
    # Parse the response as JSON
    patients = response.json()

    # Extract and print the first names from the patient data
    for entry in patients.get('entry', []):
        patient = entry.get('resource')
        for name in patient.get('name', []):
            # Assuming we want the 'usual' or 'official' names. If not, remove the next line.
            if name.get('use') in ['usual', 'official']:
                given_names = name.get('given', [])  # 'given' is a list of given names(parts)
                # Combine the parts of given name and print them
                full_given_name = ' '.join(given_names)
                print(full_given_name)
else:
    print(f'Request failed with status code: {response.status_code}')

Michelle
Sean
Julia


In [3]:

# The endpoint for searching patients by surname
patient_search_url = f'{base_url}/Patient'

# The parameters for the search query, searching for patients with the surname 'Smith'
search_params = {
    'family': 'Smith'
}

# Make the GET request to search for patients with the surname 'Smith'
response = requests.get(patient_search_url, params=search_params)

# Check if the request was successful
if response.status_code == 200:
    # Parse the response as JSON
    patients = response.json()

    # Begin constructing the markdown table
    table = ["| First Name | Last Name | Email Address | Patient ID |", "|------------|-----------|----------------|-------------|"]
    
    for entry in patients.get('entry', []):
        patient = entry.get('resource')
        patient_id = patient.get('id')
        last_name = 'Smith'  # Since we filtered by this last name
        first_names = []
        email_addresses = []

        # Extract the first names and email addresses from the patient data
        for name in patient.get('name', []):
            if name.get('use') in ['usual', 'official']:
                first_names.extend(name.get('given', []))
        
        for contact in patient.get('telecom', []):
            if contact.get('system') == 'email':
                email_addresses.append(contact.get('value'))

        # Compose the line for the current patient
        first_name = " ".join(first_names) if first_names else ""
        email_address = email_addresses[0] if email_addresses else ""
        line = f"| {first_name} | {last_name} | {email_address} | {patient_id} |"
        table.append(line)
    
    # Print the markdown table
    for line in table:
        print(line)
else:
    print(f'Request failed with status code: {response.status_code}')

| First Name | Last Name | Email Address | Patient ID |
|------------|-----------|----------------|-------------|
|  | Smith |  | 52865 |
|  | Smith |  | 53254 |
|  | Smith | sarahsmith@gmail.com | 591264 |
|  | Smith |  | 591645 |
| Michelle | Smith |  | 594843 |
|  | Smith |  | 596719 |
|  | Smith |  | 596720 |
|  | Smith |  | 596721 |
|  | Smith |  | 596722 |
|  | Smith |  | 596723 |
|  | Smith |  | 596724 |
|  | Smith |  | 596725 |
|  | Smith |  | 596726 |
|  | Smith |  | 596732 |
|  | Smith |  | 596733 |
|  | Smith |  | 596736 |
|  | Smith |  | 596737 |
|  | Smith |  | 596764 |
| Sean | Smith |  | 597226 |
| Julia | Smith |  | 618626 |


| First Name | Last Name | Email Address | Patient ID |
|------------|-----------|----------------|-------------|
|  | Smith |  | 52865 |
|  | Smith |  | 53254 |
|  | Smith | sarahsmith@gmail.com | 591264 |
|  | Smith |  | 591645 |
| Michelle | Smith |  | 594843 |
|  | Smith |  | 596719 |
|  | Smith |  | 596720 |
|  | Smith |  | 596721 |
|  | Smith |  | 596722 |
|  | Smith |  | 596723 |
|  | Smith |  | 596724 |
|  | Smith |  | 596725 |
|  | Smith |  | 596726 |
|  | Smith |  | 596732 |
|  | Smith |  | 596733 |
|  | Smith |  | 596736 |
|  | Smith |  | 596737 |
|  | Smith |  | 596764 |
| Sean | Smith |  | 597226 |
| Julia | Smith |  | 618626 |

Write a script that will search for all the patient's who are taking Alemtuzumab medication. Create a markdown table with their name, ID, the medication note.

In [10]:


# Function to search for Medication based on the display name
def search_medication(medication_name):
    response = requests.get(f"{base_url}/Medication?code:text={medication_name}")
    if response.status_code == 200:
        medications = response.json()
        # Return the ID of the first matching Medication
        if 'entry' in medications and len(medications['entry']) > 0:
            return medications['entry'][0]['resource']['id']
    return None

# Function to search for MedicationRequest based on medication ID
def get_medication_requests(medication_id):
    response = requests.get(f"{base_url}/MedicationRequest?medication={medication_id}")
    if response.status_code == 200:
        return response.json()
    return None

# Function to get Patient details by ID
def get_patient_details(patient_id):
    response = requests.get(f"{base_url}/Patient/{patient_id}")
    if response.status_code == 200:
        patient_data = response.json()
        patient_name_data = patient_data.get('name', [])[0]
        patient_name = patient_name_data.get('given', [])[0] + " " + patient_name_data.get('family', "")
        return patient_name
    return "Unknown"

# Find the ID for Alemtuzumab
medication_id = search_medication("Alemtuzumab")

# If medication_id is found, find patients taking Alemtuzumab
if medication_id:
    medication_requests = get_medication_requests(medication_id)

    # Initialize a markdown table with headers
    markdown_table = "| Name | ID | Medication Note |\n|---|---|---|\n"

    # Iterate over MedicationRequest resources and populate the markdown table
    if medication_requests and 'entry' in medication_requests:
        for entry in medication_requests['entry']:
            med_request = entry['resource']
            patient_ref = med_request['subject']['reference']
            patient_id = patient_ref.split('/')[-1]
            patient_name = get_patient_details(patient_id)
            medication_note = med_request.get('note', [{'text': ''}])[0].get('text', '')

            # Add the patient details to the markdown table
            markdown_table += f"| {patient_name} | {patient_id} | {medication_note} |\n"
else:
    markdown_table = "No Alemtuzumab medication found."

# Output the markdown table
print(markdown_table)

| Name | ID | Medication Note |
|---|---|---|

